In [1]:
import pandas as pd;
import numpy as np;
import os
import requests
import bs4 as bs
import nltk
import re
import time
import sys
import logging
import multiprocessing;
import spacy
#spacy.load('en')
from spacy.lang.en import English

In [2]:
folder_name = '/Users/michael/Downloads/SEC'
file_names = os.listdir(folder_name)
end_folder_name = '/Users/michael/Downloads/cleaned_SEC'

more_stop_words = ['company','months','products','operations','income','period','sales','market','business','stock','results',
                  'revenue','shares','share','revenues','assets','statements','value','costs','product','quarter','increase',
                  'expense','customers','performance','interest','charges','price','compensation','agreement','property','amount',
                  'markets','information','expenses','facility','customer','taxes','demand','payments',
                  'stringitemtype','textblockitemtype','monetaryitemtype','referencesreference','namespace','referencesno',
                  'definitionthe','definitionamount']

en_stop = nltk.corpus.stopwords.words('english')
en_stop.extend(more_stop_words)
parser = English()

In [3]:
def get_text_from_file(file):
    page_soup = bs.BeautifulSoup(file, 'lxml')
    divs = page_soup.find_all('div')
    all_text = ""
    for div in divs:
        text = div.getText()
        if len(text) > 200 and text[0] != '\xa0':
            ## removes all the /xa0 /n /t garbage and some extra spaces
            cleaner_text = re.sub('\s+', ' ', text).strip()
            all_text = all_text + " " + cleaner_text            
    return all_text

def tokenize(text):
    '''
    Separates each word from the others, making them lower case and removing URLs and possible screen names that show up.
    '''

    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def combine_lists(list_of_lists):
    out = []
    for li in list_of_lists:
        out.append(li)
    return out

In [4]:
def clean_filing(start_folder,file_name,end_folder):
    
    ## get the raw file
    filing = open(start_folder + '/' + file_name).read()
    text = get_text_from_file(filing)
    
    if len(text)> 10**6 - 1 :
        #inc = 10**6
        #texts = [text[(i-1) * inc : i * inc] for i in range(0, text, 10**6)]
        #tokens = combine_lists(texts)
        text1 = text[:999999]
        text2 = text[999999:]
        tokens = tokenize(text1) + tokenize(text2)
    else:
        tokens = tokenize(text)
        
    new_tokens = []
    for tok in tokens:
        if len(tok) > 4 and len(tok) < 14 and tok[0] not in {'-', 'x'} and tok not in en_stop:
            new_tokens.append(tok)
            
    pos_tags = nltk.pos_tag(new_tokens)
    
    final_text = ""
    for pos_tag in pos_tags:
        if pos_tag[1][0] == "N":
            final_text = final_text + " " + pos_tag[0]

    full_file_name = end_folder + "/" + file_name
    text_file = open(full_file_name, "w+")
    text_file.write(final_text)
    text_file.close()

In [5]:
test = clean_filing(folder_name,file_names[0],end_folder_name)